# Classification   $ $

<hr>
<em> Solving the classification problem of EEG signals associted with a breaking event.<br>
First the signals are divided in sub intervales and average over the EEG channels. <br>
We compute differents statistics over the average signal.<br>
The we input thee extract features from the series and use them with normal supervised learning.
</em> 
<hr>

In [11]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from advisedlib.extraction.csv import CsvExtractor
from advisedlib.dataframe import SynchronousPandasDataFrame, AsynchronousPandasDataFrame, StaticPandasDataFrame
from advisedlib.feature_selection.ls import LS
from advisedlib.feature_extraction.tsfresh_feature_extractor import Tsfresh
from tsfresh.feature_extraction import ComprehensiveFCParameters, EfficientFCParameters
from tsfresh import extract_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh import extract_features, select_features

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression as LR
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.metrics import roc_auc_score as AUC, accuracy_score as accuracy

In [2]:
path_to_data = os.path.normpath(os.path.join('/Users/obando_c/Documents/Neuro/EmergencyBraking/data/post_processed/Balanced_01'))
files = sorted([elt for elt in os.listdir(path_to_data) if elt[-4:] == '.npy'],
                   key=(lambda x: int(x.split('_')[2])))
np.random.RandomState(seed=0).shuffle(files)
eeg_channels_names = ['Fp1', 'Fp2', 'AF3', 'AF4', 'F7', 'F5', 'F3', 'F1', 'Fz', 'F2', 'F4', 'F6', 'F8', 'FT7', 
 'FC5', 'FC3', 'FC1', 'FCz', 'FC2', 'FC4', 'FC6', 'FT8', 'T7', 'C5', 'C3', 'C1', 'Cz', 'C2', 
 'C4', 'C6', 'T8', 'TP7', 'CP5', 'CP3', 'CP1', 'CPz', 'CP2', 'CP4', 'CP6', 'TP8', 'P9', 'P7',
 'P5', 'P3', 'P1', 'Pz', 'P2', 'P4', 'P6', 'P8', 'P10', 'PO7', 'PO3', 'POz', 'PO4', 'PO8', 
 'O1', 'Oz', 'O2']

In [3]:
# DO not run
df_eeg_mean = pd.DataFrame([])
labels = []
time_names = ['t'+ str(x) for x in range(319 + 1)]
for num, file in enumerate(files): # create a pandas df with 
    label = {'event': 1.0, 'noevent': 0.0}[file.split('_')[-1].split('.')[0]]
    data = np.load(os.path.join(path_to_data,file))
    df_interval = pd.DataFrame(data = data.mean(0).reshape(1,320))
    df_interval.insert(0, "seq_id", num, True) 
    df_interval.insert(1, "event", label, True) 
    df_interval.insert(2, "param", 'eeg_mean', True) 
    df_interval.columns = ["seq_id", "event", "param"] + time_names
    
    df_eeg_mean = df_eeg_mean.append(df_interval)
    
    labels.append(label)
df_eeg_mean.to_csv(r'df_eeg_mean.csv', index = False)

In [4]:
filepath = "df_eeg_mean.csv"
csvextractor = CsvExtractor(
    advised_pandas_dataframe_class=SynchronousPandasDataFrame,
    filepath_or_buffer=filepath,
    kwargs_pandas_read_csv={},
    context_columns=["seq_id", "event"],
    context_id_columns=["seq_id"],
    #time_mesh= NPtime.T, #4.0e-06  # ns
    sampling_rate = 200, # 4 Like in tutorial
    #sampling_period = 5.0e-03 #
)
sync_df, context_df = csvextractor.extract()

display(sync_df)
display(context_df)

,seq_id,param,t0,t1,t2,t3,t4,t5,t6,t7,...,t310,t311,t312,t313,t314,t315,t316,t317,t318,t319
0,0,eeg_mean,-6.662516,-8.375671,-9.692421,-11.467716,-13.143236,-12.975047,-9.707634,-4.195401,...,-22.263763,-13.954300,-11.058272,-16.370016,-27.041126,-36.342974,-39.618671,-38.254451,-37.336343,-39.089531
1,1,eeg_mean,-2.805284,-2.307130,-5.302989,-11.020308,-16.297534,-18.242868,-16.343058,-12.936401,...,-9.855006,-5.390771,0.895795,6.875656,9.662323,8.794505,6.693420,5.872191,5.934004,4.577942
2,2,eeg_mean,-1.290911,-2.031751,-3.469748,-5.174402,-5.608096,-3.689727,-0.252362,2.304056,...,9.826430,9.240839,9.264046,9.858784,9.633315,7.927111,5.669283,4.137048,3.243828,1.676554
3,3,eeg_mean,4.864749,1.798598,-1.390380,-2.387058,-1.963290,-1.977039,-2.528718,-1.790300,...,-9.151047,-11.481513,-12.562011,-11.573424,-9.808627,-9.210363,-10.059707,-10.917036,-10.287159,-8.389499
4,4,eeg_mean,-4.099958,-5.756953,-5.614080,-4.656812,-3.873650,-3.240921,-2.185156,-0.863298,...,7.860007,9.911851,8.771263,5.549002,2.413375,0.576752,0.045686,0.415853,1.342473,2.199935
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
455,455,eeg_mean,8.322151,8.059048,7.217260,6.392138,5.434953,3.959081,1.299081,-3.124878,...,-11.840574,-11.258744,-9.702749,-11.919377,-17.940323,-22.461207,-21.160908,-15.664134,-11.067216,-10.157483
456,456,eeg_mean,-9.614742,-8.875775,-8.767794,-9.633213,-10.340118,-9.019968,-5.077338,-0.080666,...,4.007202,5.288535,7.832677,11.704519,15.497023,17.081570,15.266195,11.166261,7.549430,6.357839
457,457,eeg_mean,0.491359,-1.100050,-1.584725,-1.441249,-1.918843,-2.454396,-0.918144,3.141293,...,8.390528,-0.326728,-2.939093,1.969193,8.163650,8.797872,3.966998,-0.062077,1.487164,6.744657
458,458,eeg_mean,8.100704,7.049934,4.305268,2.108500,0.980204,-0.301559,-2.268085,-3.632058,...,-4.380859,-3.042511,-0.896583,-1.488917,-4.851205,-7.294311,-5.968771,-2.699228,-1.563740,-4.107173


,seq_id,event
0,0,1.0
1,1,0.0
2,2,1.0
3,3,1.0
4,4,0.0
...,...,...
455,455,1.0
456,456,0.0
457,457,1.0
458,458,0.0


In [5]:
print(sync_df.shape)
#print(df_eeg.T.shape)
#print(sync_df.time_mesh)
len(sync_df.time_mesh)

(460, 322)


320

In [6]:
params = sync_df.param.drop_duplicates().tolist()
print(params)
assert len(params) == 1

['eeg_mean']


In [7]:
sync_df.check()

,seq_id,param,t0,t1,t2,t3,t4,t5,t6,t7,...,t310,t311,t312,t313,t314,t315,t316,t317,t318,t319
0,0,eeg_mean,-6.662516,-8.375671,-9.692421,-11.467716,-13.143236,-12.975047,-9.707634,-4.195401,...,-22.263763,-13.954300,-11.058272,-16.370016,-27.041126,-36.342974,-39.618671,-38.254451,-37.336343,-39.089531
1,1,eeg_mean,-2.805284,-2.307130,-5.302989,-11.020308,-16.297534,-18.242868,-16.343058,-12.936401,...,-9.855006,-5.390771,0.895795,6.875656,9.662323,8.794505,6.693420,5.872191,5.934004,4.577942
2,2,eeg_mean,-1.290911,-2.031751,-3.469748,-5.174402,-5.608096,-3.689727,-0.252362,2.304056,...,9.826430,9.240839,9.264046,9.858784,9.633315,7.927111,5.669283,4.137048,3.243828,1.676554
3,3,eeg_mean,4.864749,1.798598,-1.390380,-2.387058,-1.963290,-1.977039,-2.528718,-1.790300,...,-9.151047,-11.481513,-12.562011,-11.573424,-9.808627,-9.210363,-10.059707,-10.917036,-10.287159,-8.389499
4,4,eeg_mean,-4.099958,-5.756953,-5.614080,-4.656812,-3.873650,-3.240921,-2.185156,-0.863298,...,7.860007,9.911851,8.771263,5.549002,2.413375,0.576752,0.045686,0.415853,1.342473,2.199935
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
455,455,eeg_mean,8.322151,8.059048,7.217260,6.392138,5.434953,3.959081,1.299081,-3.124878,...,-11.840574,-11.258744,-9.702749,-11.919377,-17.940323,-22.461207,-21.160908,-15.664134,-11.067216,-10.157483
456,456,eeg_mean,-9.614742,-8.875775,-8.767794,-9.633213,-10.340118,-9.019968,-5.077338,-0.080666,...,4.007202,5.288535,7.832677,11.704519,15.497023,17.081570,15.266195,11.166261,7.549430,6.357839
457,457,eeg_mean,0.491359,-1.100050,-1.584725,-1.441249,-1.918843,-2.454396,-0.918144,3.141293,...,8.390528,-0.326728,-2.939093,1.969193,8.163650,8.797872,3.966998,-0.062077,1.487164,6.744657
458,458,eeg_mean,8.100704,7.049934,4.305268,2.108500,0.980204,-0.301559,-2.268085,-3.632058,...,-4.380859,-3.042511,-0.896583,-1.488917,-4.851205,-7.294311,-5.968771,-2.699228,-1.563740,-4.107173


In [8]:
context_df.event.value_counts()

0.0    230
1.0    230
Name: event, dtype: int64

## Feature extraction: tsfresh + selector
[Tsfresh](https://tsfresh.readthedocs.io) is a python library that is designed to extract automatically a lot of features from a timeseries. 

A wrapper for tsfresh extractor is embedded in advisedlib. It can be coupled with a feature selection step in order to reduce the number of generated features and to be able to handle them afterwards with an anomaly scorer.

### Conversion to asynchronous format
The tsfresh wrapper needs an asynchronous input, so we first convert the dataset to asynchronous format.
See the [cookbook](https://docs.cognitive.bocacc.io/advised-lib/notebooks/cookbook.html#Conversion-between-advised-formats) for an example.

In [9]:
async_df = sync_df.to_asynchronous()
display(async_df)

,seq_id,param,time,value
0,0,eeg_mean,2000-01-01 00:00:00.000,-6.662516
1,0,eeg_mean,2000-01-01 00:00:00.005,-8.375671
2,0,eeg_mean,2000-01-01 00:00:00.010,-9.692421
3,0,eeg_mean,2000-01-01 00:00:00.015,-11.467716
4,0,eeg_mean,2000-01-01 00:00:00.020,-13.143236
...,...,...,...,...
147195,459,eeg_mean,2000-01-01 00:00:01.575,30.539376
147196,459,eeg_mean,2000-01-01 00:00:01.580,28.284688
147197,459,eeg_mean,2000-01-01 00:00:01.585,25.063752
147198,459,eeg_mean,2000-01-01 00:00:01.590,23.900855


In [10]:
f = extract_features( async_df, column_id = "seq_id", column_sort = "time", column_kind='param', column_value="value" )

Feature Extraction: 100%|██████████| 20/20 [00:39<00:00,  1.95s/it]


In [65]:
impute( f )
assert f.isnull().sum().sum() == 0

In [66]:
features_file = "df_eeg_mean_features.csv"

In [67]:
f['y'] = labels
f.to_csv( features_file, index = None )

In [68]:
features = pd.read_csv( features_file )

In [69]:
validation_split_i = int(features.shape[0]/2)

In [70]:
train_x = features.iloc[:validation_split_i].drop( 'y', axis = 1 )
test_x = features.iloc[validation_split_i:].drop( 'y', axis = 1 )

train_y = features.iloc[:validation_split_i].y
test_y = features.iloc[validation_split_i:].y

In [71]:
train_features_selected = select_features( train_x, train_y, fdr_level = 0.05 )

In [74]:
print(train_features_selected.columns)

Index(['eeg_mean__spkt_welch_density__coeff_5',
       'eeg_mean__autocorrelation__lag_1',
       'eeg_mean__partial_autocorrelation__lag_1',
       'eeg_mean__spkt_welch_density__coeff_2',
       'eeg_mean__autocorrelation__lag_2', 'eeg_mean__autocorrelation__lag_3',
       'eeg_mean__agg_linear_trend__f_agg_"mean"__chunk_len_10__attr_"stderr"',
       'eeg_mean__agg_linear_trend__f_agg_"min"__chunk_len_10__attr_"stderr"',
       'eeg_mean__approximate_entropy__m_2__r_0.5',
       'eeg_mean__cid_ce__normalize_True',
       'eeg_mean__agg_linear_trend__f_agg_"min"__chunk_len_50__attr_"stderr"',
       'eeg_mean__approximate_entropy__m_2__r_0.3',
       'eeg_mean__fft_coefficient__coeff_8__attr_"abs"',
       'eeg_mean__autocorrelation__lag_4',
       'eeg_mean__longest_strike_below_mean',
       'eeg_mean__approximate_entropy__m_2__r_0.7',
       'eeg_mean__approximate_entropy__m_2__r_0.9', 'eeg_mean__variance',
       'eeg_mean__standard_deviation',
       'eeg_mean__agg_linear_trend_

In [75]:
train = train_features_selected.copy()
train['y'] = train_y

test = test_x[ train_features_selected.columns ].copy()
test['y'] = test_y

In [76]:
train_file = "df_eeg_mean_train.csv"
test_file = "df_eeg_mean_test.csv"

In [77]:
train.to_csv( train_file, index = None )
test.to_csv( test_file, index = None )

In [78]:
train = pd.read_csv( train_file )
test = pd.read_csv( test_file )

x_train = train.drop( 'y', axis = 1 ).values
y_train = train.y.values

x_test = test.drop( 'y', axis = 1 ).values
y_test = test.y.values

In [12]:
classifiers = [
#LR( C = 10 ),
#LR( C = 1 ),
#LR( C = 0.1 ),
    
#make_pipeline( StandardScaler(), LR()),	
#make_pipeline( StandardScaler(), LR( C = 10 )),
make_pipeline( StandardScaler(), LR( C = 30 )),

#make_pipeline( MinMaxScaler(), LR()),
#make_pipeline( MinMaxScaler(), LR( C = 10 )),	
make_pipeline( MinMaxScaler(), LR( C = 30 )),

#LDA(),
RF( n_estimators = 100, min_samples_leaf = 5 )
]

In [82]:
for clf in classifiers:
    clf.fit( x_train, y_train )
    p = clf.predict_proba( x_test )[:,1]
    p_bin = clf.predict( x_test )
    
    auc = AUC( y_test, p )
    acc = accuracy( y_test, p_bin )
    print( "AUC: {:.2%}, accuracy: {:.2%} \n\n{}\n\n".format( auc, acc, clf ))


/Users/obando_c/anaconda3/envs/advisedlib-env/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/Users/obando_c/anaconda3/envs/advisedlib-env/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://

AUC: 79.47%, accuracy: 74.35% 

Pipeline(memory=None,
         steps=[('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('logisticregression',
                 LogisticRegression(C=30, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
                                    l1_ratio=None, max_iter=100,
                                    multi_class='auto', n_jobs=None,
                                    penalty='l2', random_state=None,
                                    solver='lbfgs', tol=0.0001, verbose=0,
                                    warm_start=False))],
         verbose=False)


AUC: 76.58%, accuracy: 71.74% 

Pipeline(memory=None,
         steps=[('minmaxscaler', MinMaxScaler(copy=True, feature_range=(0, 1))),
                ('logisticregression',
                 LogisticRegression(C=30, class_weight=None, dual=False,
                                    fi

In [ ]:
# The NN does a better job in terms of accuracy 

In [ ]:
# What if I collapse all the electrodes into one cirtual one and then extract features in thet signal